# Imports

In [ ]:
!pip install emoji
!pip install demoji
!pip install langdetect

In [ ]:
import pandas as pd
import numpy as np

import time
import re
from string import punctuation


import demoji
demoji.download_codes()

... OK (Got response in 0.17 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


# Tweets

In [ ]:
data = pd.read_excel("dev_data_tweet.xlsx")
data.head()

,Tweet_ID,Tweet,Mobile_Tech_Tag
0,tweet_0001,You'll 💜 my #PitchWars book if you like: 🦋 hat...,0
1,tweet_0002,RT @SkySportsNews: 🚨 Breaking: #WBA have reach...,0
2,tweet_0003,RT @stealyoman_cuso: really says a lot about s...,0
3,tweet_0004,RT @PGtzsche1: HPV vaccines increased serious ...,0
4,tweet_0005,Ramaphosa says if you are positive you must se...,0


In [ ]:
data = data.drop("Tweet_ID", axis=1)

In [ ]:
data.head()

,Tweet,Mobile_Tech_Tag
0,You'll 💜 my #PitchWars book if you like: 🦋 hat...,0
1,RT @SkySportsNews: 🚨 Breaking: #WBA have reach...,0
2,RT @stealyoman_cuso: really says a lot about s...,0
3,RT @PGtzsche1: HPV vaccines increased serious ...,0
4,Ramaphosa says if you are positive you must se...,0


In [ ]:
data.isnull().sum()

Tweet              0
Mobile_Tech_Tag    0
dtype: int64

## Preprocessing the Tweets

In [ ]:
sample_text = data['Tweet'][0]
print(sample_text)

You'll 💜 my #PitchWars book if you like: 🦋 hate-to-love 🦋 #OwnVoices Ace/Aro rep 🦋 subdued mythology 🦋 ride-or-die queerplatonic bonds 🦋 fast pace, high risk 🦋 star and plant imagery everywhere 🦋 getting punched in your emotions 🌿#PWParty20⏱️ https://t.co/mDTSG4DLG5


In [ ]:
def contraction_mapping(text):
  contraction_map = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
                       "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                       "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                       "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am",
                       "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                       "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                       "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                       "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                       "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                       "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
                       "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would",
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have",
                       "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                       "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                       "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
                       "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", 
                       "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                       "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have",
                       "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'u.s':'america', 'e.g':'for example'}
  for word in contraction_map.keys():
    text = text.replace(word, contraction_map[word])
  return text

In [ ]:
def clear_punk(text):
  punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-",
                 "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 
                 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '!':' '}
  punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

  for p in punct_mapping:
    text = text.replace(p, punct_mapping[p])

  for p in punct:
    text = text.replace(p,' ')
  
  return text

In [ ]:
def hashtag_seg(tweet):
  # segmenter using the word statistics from Twitter
  seg_tw = Segmenter(corpus="twitter",)

  tweet_lst = tweet.split(' ')
  new_tweet_lst = []
  for word in tweet_lst:
    if word[0] == '#':
      word = re.sub(r'#','', word)
      word = seg_tw.segment(word)
    
    new_tweet_lst.append(word)
  
  tweet = ' '.join(new_tweet_lst)
  return tweet

In [ ]:
def cleaner(tweet):
  # add space before and after
  tweet = ' '+tweet+' '

  # remove useless words
  tweet = recycle_bin(tweet)

  # to replace emoji with
  tweet = demoji.replace_with_desc(tweet)
  
  # URL
  tweet = re.sub(r"https?:\/\/t.co\/[A-Za-z0-9]+", "", tweet)

  # seperate words in a hashtag
  tweet = re.sub( r"(?<=\w)([A-Z])", r" \1", tweet)  
  
  # lower case
  tweet = tweet.lower()
  
  # expanding the contractions
  tweet = contraction_mapping(tweet)

  # punctuations 
  # for punk in punctuation:
  #   tweet = tweet.replace(punk," ")
  tweet = clear_punk(tweet)


  # remove single alphabets
  # tweet = re.sub(r' [a-z] ', ' ', tweet)

  # remove excess space
  tweet = re.sub(r' +', ' ', tweet)

  return tweet

In [ ]:
cleaner(sample_text)

' you will purple heart my pitch wars book if you like butterfly hate to love butterfly own voices ace aro rep butterfly subdued mythology butterfly ride or die queerplatonic bonds butterfly fast pace high risk butterfly star and plant imagery everywhere butterfly getting punched in your emotions herb p w party20 stopwatch '

# Translated texts

In [ ]:
!pip install mtranslate

  Created wheel for mtranslate: filename=mtranslate-1.8-cp37-none-any.whl size=3696 sha256=82d4c357bd6d31586fb0d82fe6073400ddd205b914e082f28eb8a24baa3f6141
  Stored in directory: /root/.cache/pip/wheels/eb/fb/4a/f63e74cbfb835161f3f2d1a6b607b137d344a5cb6d8c1303fa
Successfully built mtranslate


In [ ]:
import mtranslate

In [ ]:
tweets = list(data["Tweet"])

In [ ]:
cleaned_sents = [cleaner(text) for text in tweets]

KeyboardInterrupt: ignored

In [ ]:
translated_sents = [mtranslate.translate(text) for text in cleaned_sents]
# translated_sents

In [ ]:
data['cleaned_tweet'] = cleaned_sents
data['translated_tweet'] = translated_sents

In [ ]:
data.head()

,Tweet,Mobile_Tech_Tag,cleaned_tweet,translated_tweet
0,You'll 💜 my #PitchWars book if you like: 🦋 hat...,0,you will purple heart my pitch wars book if y...,you will purple heart my pitch wars book if yo...
1,RT @SkySportsNews: 🚨 Breaking: #WBA have reach...,0,sky sports news police car light breaking w b...,sky sports news police car light breaking w b ...
2,RT @stealyoman_cuso: really says a lot about s...,0,stealyoman cuso really says a lot about socie...,stealyoman cuso really says a lot about societ...
3,RT @PGtzsche1: HPV vaccines increased serious ...,0,p gtzsche1 h p v vaccines increased serious n...,p gtzsche1 h p v vaccines increased serious ne...
4,Ramaphosa says if you are positive you must se...,0,ramaphosa says if you are positive you must s...,ramaphosa says if you are positive you must se...


In [ ]:
data.to_excel("dev_data_tweet_cleaned_v4.xlsx")

In [ ]:
i=7
print(tweets[i])
print(cleaned_sents[i])

@EdwardthembaSa 😂 I don't think hz taking that risk, across the limpopo it rumoured that senior politicians took the jab in secret and didn't turn out well.
 edwardthemba sa face with tears of joy i don t think hz taking that risk across the limpopo it rumoured that senior politicians took the jab in secret and didn t turn out well 


# Experiments

## Lang Detect

In [ ]:
from langdetect import detect_langs

In [ ]:
start = time.time()

for tweet in data['Tweet']:
  detect_langs(tweet)

end = time.time()

print(end - start)

33.57750630378723


## iNLTK Library

In [ ]:
!pip install inltk

     |████████████████████████████████| 81kB 3.0MB/s 
     |████████████████████████████████| 1.2MB 5.4MB/s 
     |████████████████████████████████| 1.3MB 17.0MB/s 
     |████████████████████████████████| 235kB 22.7MB/s 
     |████████████████████████████████| 296kB 23.0MB/s 
     |████████████████████████████████| 143kB 24.0MB/s 
  Created wheel for typing: filename=typing-3.7.4.3-cp37-none-any.whl size=26308 sha256=e6b82743a26a4076359acf30bd87d9e0670983535f0a6420cd7c0b4e7f28d4fc
  Stored in directory: /root/.cache/pip/wheels/2d/04/41/8e1836e79581989c22eebac3f4e70aaac9af07b0908da173be
Successfully built typing
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


## Google Translate

In [ ]:
!pip install googletrans==3.1.0a0

In [ ]:
from googletrans import Translator
translator = Translator()

In [ ]:
def trans_to_english(tweet):
  translated_tweet = translator.translate(tweet, dest='en')
  return translated_tweet.text

In [ ]:
trans_to_english(" BanAllLoanApps जब देश का युवा और मध्यमवर्ग सरकार से जोरो से गुहार लगाने लगे तो समझ लेना चाहिए मामला गंभीर हो चला है जी हा मै fraud chinies loan appsकी बात कर रहा हू सरकार को अब नींद से जाग जाना चाहिए बहोत हुआ अब rbi को action लेने का समय आ गया है    PMOIndia  ianuragthakur  RBI")

'BanAllLoanApps When the youth and middle class of the country start pleading with the government, then it should be understood that the matter has become serious, I am talking about fraud chinies loan apps, the government should wake up from the sleep now and now to take action to rbi The time has come for PMOIndia ianuragthakur RBI'

In [ ]:
tweets = list(data["Tweet"])

In [ ]:
data['cleaned_tweet'] = [trans_to_english(preprocessing(text)) for text in tweets] 

In [ ]:
data = data.drop('Tweet', axis=1)
data.head()

,Tweet_ID,Mobile_Tech_Tag,cleaned_tweet
0,tweet_0001,0,You ll purple heart my PitchWars book if yo...
1,tweet_0002,0,RT SkySportsNews police car light Breaking...
2,tweet_0003,0,RT stealyoman cuso really says a lot about s...
3,tweet_0004,0,RT PGtzsche1 HPV vaccines increased serious ...
4,tweet_0005,0,Ramaphosa says if you are positive you must se...


In [ ]:
data.to_excel("dev_data_tweet_cleaned.xlsx")

## MTranslate

In [ ]:
!pip install mtranslate

  Created wheel for mtranslate: filename=mtranslate-1.8-cp37-none-any.whl size=3696 sha256=2ef2fadc98cb55c2066fbfdbb72feceaafb205fe71e9957da7053ed2c4c10dee
  Stored in directory: /root/.cache/pip/wheels/eb/fb/4a/f63e74cbfb835161f3f2d1a6b607b137d344a5cb6d8c1303fa
Successfully built mtranslate


In [ ]:
import mtranslate

In [ ]:
tweets = list(data["Tweet"])

In [ ]:
translated_sents = [mtranslate.translate(text) for text in tweets]
# translated_sents

In [ ]:
cleaned_sents = [cleaner(text) for text in translated_sents]

In [ ]:
data['cleaned_tweet'] = cleaned_sents

In [ ]:
data.head()

,Tweet,Mobile_Tech_Tag,cleaned_tweet
0,You'll 💜 my #PitchWars book if you like: 🦋 hat...,0,you ll purple heart my pitch wars book if you ...
1,RT @SkySportsNews: 🚨 Breaking: #WBA have reach...,0,r t sky sports news police car light breaking ...
2,RT @stealyoman_cuso: really says a lot about s...,0,r t stealyoman cuso really says a lot about so...
3,RT @PGtzsche1: HPV vaccines increased serious ...,0,r t p gtzsche1 h pv vaccines increased serious...
4,Ramaphosa says if you are positive you must se...,0,ramaphosa says if you are positive you must se...


In [ ]:
data.to_excel("dev_data_tweet_cleaned_v3.xlsx")

In [ ]:
cleaned_sents[1]

'r t sky sports news police car light breaking w ba have reached a verbal agreement with a fc in the last hour over a loan deal for ainsley maitland niles rob dorsett sky adds he will undergo a medical in london in the next couple of hours deadline day handshake '

## Google Transliteration

In [ ]:
!pip install google-transliteration-api

In [ ]:
from google.transliteration import transliterate_word
start = time.time()
suggestions = transliterate_word(data['Tweet'][1811], lang_code='hi')
end = time.time()
print(suggestions[0])
print(end-start)

@अबक्कारे आदित्य बिरला कंपनी फ्रॉड ८४०० से रुपये मुझसे लेकर लोन अप्रूवल अभी तक नहीं किया एक महीना हो गया किसी गरीब का पैसा लूट हैं कृपया सर्कार से निवेदन है आदित्य
0.07883429527282715


## Aksharamukha

In [ ]:
!pip install aksharamukha

     |████████████████████████████████| 163kB 4.3MB/s 


In [ ]:
from aksharamukha import transliterate

In [ ]:
transliterate.auto_detect("अभिलाष")

'Devanagari'

In [ ]:
start = time.time()
print(transliterate.process('HK', 'Devanagari', data['Tweet'][1811]))
end = time.time()

print(data['Tweet'][1811])
print(end - start)

@अब्च्चरे आदित्य Bइर्ल चोम्पन्य् फ़्रौद् ८४०० से रुपये मुझ्से लेकर् लोअन् अप्प्रोवल् अभि तक् नहिन् किय एक् ंअहिन ःओ ङय किसि ङरिब् क Pऐस लोओत् ःऐन् क्रिप्य षर्कर् से निवेदन् है आदित्य Bइर्ल चोम्पन्य् को बन्द् करय जअये सेअल् कर् दिय जअये और् मेदिअ wअले भि सुन्ते नहिन् मुसिच् य़ोउटुबे ह्त्त्प्स्://त्।चो/Xxफ़्स्घ्Wग्ज्५
@abccare Aditya Birla company fraud 8400 se rupaye mujhse lekar loan approval abhi tak nahin kiya ek Mahina Ho Gaya kisi Garib ka Paisa loot Hain kripya Sarkar se nivedan hai Aditya Birla company ko band karaya jaaye seal kar diya jaaye aur media wale bhi sunte nahin music YouTube https://t.co/XxfsghWgj5
0.013390064239501953


## Ekhprasis